In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping


In [2]:
df=pd.read_csv('tesla_clean.csv')

In [3]:
df.head()

,Date,Close
0,2019-05-13,45.402000
1,2019-05-14,46.462002
2,2019-05-15,46.389999
3,2019-05-16,45.666000
4,2019-05-17,42.206001


In [4]:
df.set_index('Date', inplace=True)

In [24]:
training_set = df.iloc[:450,].values
test_set = df.iloc[450:,].values

In [42]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, 450):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
#0 o dong column, de tao cac vector thanh nhung dong rieng biet
X_train, y_train = np.array(X_train), np.array(y_train)

Reshape the data to be 3-dimensional in the form [number of samples, number of time steps, and number of features]. The LSTM model is expecting a 3-dimensional data set.

In [49]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [50]:
X_train.shape

(390, 60, 1)

In [51]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
13/13 [==============================] - 8s 108ms/step - loss: 0.1341
Epoch 2/100
13/13 [==============================] - 1s 102ms/step - loss: 0.0103
Epoch 3/100
13/13 [==============================] - 1s 100ms/step - loss: 0.0061
Epoch 4/100
13/13 [==============================] - 1s 103ms/step - loss: 0.0055
Epoch 5/100
13/13 [==============================] - 1s 102ms/step - loss: 0.0062
Epoch 6/100
13/13 [==============================] - 1s 99ms/step - loss: 0.0036
Epoch 7/100
13/13 [==============================] - 1s 101ms/step - loss: 0.0073
Epoch 8/100
13/13 [==============================] - 1s 100ms/step - loss: 0.0038
Epoch 9/100
13/13 [==============================] - 1s 97ms/step - loss: 0.0043
Epoch 10/100
13/13 [==============================] - 1s 100ms/step - loss: 0.0063
Epoch 11/100
13/13 [==============================] - 1s 97ms/step - loss: 0.0056
Epoch 12/100
13/13 [==============================] - 1s 102ms/step - loss: 0.0063
Epoch 13/100
13/

13/13 [==============================] - 1s 102ms/step - loss: 0.0024
Epoch 100/100
13/13 [==============================] - 2s 116ms/step - loss: 0.0020


(390,)